# **💧 CleanWater.AI – Part 2: Predicting Potable Water**

This notebook loads the trained model from Part 1 and uses it to predict if a new sample of water is safe to drink.

Model trained with chemical water data (pH, solids, sulfate, etc.).

Part of UN SDG 6 – Clean Water and Sanitation.


**1.Import Libraries**

In [50]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from google.colab import files
uploaded = files.upload()


Saving water_model (1).pt to water_model (1) (4).pt


**2. Defining the Neural Network Class**

In [51]:
# 2. Define the Neural Network Class (Must match Part 1 model architecture)
class WaterNet(nn.Module):
    def __init__(self, input_dim):
        super(WaterNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


**3. Loading the Trained Model**

In [52]:
# 3. Loading the Trained Model
model = WaterNet(input_dim=9)
model.load_state_dict(torch.load("water_model (1).pt",weights_only=False))
model.eval()
print("✅ Model loaded successfully!")


✅ Model loaded successfully!


**4. Prepare Data**

In [53]:
# 4. Prepare Data
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("water_potability (1).csv")
print(df.head())

df = pd.read_csv("water_potability (1).csv")
df.fillna(df.mean(), inplace=True)

X = df.drop("Potability", axis=1)

scaler = StandardScaler()
scaler.fit(X)

Saving water_potability (1).csv to water_potability (1) (1).csv
         ph    Hardness        Solids  Chloramines     Sulfate  Conductivity  \
0       NaN  204.890455  20791.318981     7.300212  368.516441    564.308654   
1  3.716080  129.422921  18630.057858     6.635246         NaN    592.885359   
2  8.099124  224.236259  19909.541732     9.275884         NaN    418.606213   
3  8.316766  214.373394  22018.417441     8.059332  356.886136    363.266516   
4  9.092223  181.101509  17978.986339     6.546600  310.135738    398.410813   

   Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       10.379783        86.990970   2.963135           0  
1       15.180013        56.329076   4.500656           0  
2       16.868637        66.420093   3.055934           0  
3       18.436524       100.341674   4.628771           0  
4       11.558279        31.997993   4.075075           0  


StandardScaler()

**5. Prediction Function**

In [54]:
# 5. Prediction Function
'''
Input: List of 9 water feature values
Output: Potability prediction + confidence

'''

def predict_potability(features):

    # Convert input list to numpy array and reshape for scaler
    input_data = np.array(features).reshape(1, -1)

    # Scale the data using fitted scaler
    input_scaled = scaler.transform(input_data)

    # Convert scaled data to torch tensor
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32)

    # Predict with model
    with torch.no_grad():
        prediction = model(input_tensor).item()

    # Return readable result
    if prediction > 0.5:
        return f"✅ Potable (Safe to drink) — Confidence: {prediction:.2f}"
    else:
        return f"🚫 Not Potable — Confidence: {prediction:.2f}"

**6. Example Usage**

In [55]:
# 6. Example Usage: A water sample with 9 feature values
sample = [7.2, 180.0, 15000, 6.5, 300.0, 420.0, 12.0, 80.0, 4.2]

print("Prediction for Sample Water:")
print(predict_potability(sample))


Prediction for Sample Water:
🚫 Not Potable — Confidence: 0.33


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
